In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
cd drive/My\ Drive/빅데이터/Bigdata_Python_MachineLearning/JEJU

/content/drive/My Drive/빅데이터/Bigdata_Python_MachineLearning/JEJU


신용카드 사용 내역 데이터(2019.01 ~ 2020.03)를 활용한 지역별, 업종별 월간 카드 사용 총액(2020.04) 예측

# 데이터셋 이해

REG_YYMM : 년월

CARD_SIDO_NM : 카드이용지역_시도 (가맹점 주소 기준)

CARD_CCG_NM : 카드이용지역_시군구 (가맹점 주소 기준)

STD_CLSS_NM : 업종명

HOM_SIDO_NM : 거주지역_시도, (고객 집주소 기준)

HOM_CCG_NM : 거주지역_시군구 (고객 집주소 기준)

AGE : 연령대

SEX_CTGO_CD : 성별 (1: 남성, 2: 여성)

FLC : 가구생애주기 (1: 1인가구, 2: 영유아자녀가구, 3: 중고생자녀가구, 4: 성인자녀가구, 5: 노년가구)

CSTMR_CNT : 이용고객수 (명)

AMT : 이용금액 (원)

CNT : 이용건수 (건)

# 데이터 전처리

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

In [6]:
data = pd.read_csv('201901-202003.csv')
data.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


In [3]:
# 날짜 처리
# REG_YYMM 컬럼은 8자리 숫자 형식으로 되어 있으므로 각각 연도와 월로 나눈다.

def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [7]:
data = data.fillna('')
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)

In [8]:
# 베이스라인

df = data.copy()

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM','HOM_CCG_NM','CARD_CCG_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [ ]:
# For문을 원하는 월 내의 지역, 업종으로 한정시킬 때 사용
# group_n = df[(df['year']==2020) & (df['month']==3)]
# group_n

In [9]:
group = df.groupby(['CARD_SIDO_NM','STD_CLSS_NM']).sum()
df1 = df.set_index(['CARD_SIDO_NM','STD_CLSS_NM'])

# EDA

- 대회 목표는 지역별 업종별 AMT 예측
- 업종별로 코로나-19의 영향력은 상이할 것

- 국내 코로나 첫 발병은 1월 20일
- 확진자 폭증의 원인인 신천지 31번째 환자 확진 2월 18일

In [ ]:
# 연도, 월 별 '이용금액'과 '이용건수'
def month(a, b):
    data1 = df[df['year']==a]
    c = data1[data1['month']==b]
    d = c.groupby(['year','month','STD_CLSS_NM']).agg({'AMT':'sum','CNT':'sum'})
    e = d.reset_index()
    return e

# 전 월 대비 '이용금액'과 '이용건수'의 변동률
# AMT_dif, CNT_dif : 전 월 대비 변동률
def dif_month(x, y):
    def month(x, y):
        data1 = df[df['year']==x]
        c = data1[data1['month']==y]
        d = c.groupby(['year','month','STD_CLSS_NM']).agg({'AMT':'sum','CNT':'sum'})
        p = d.reset_index()
        return p
    i = y-1
    e = month(x, y)
    o = month(x, i)
    c = pd.DataFrame((e['AMT'] - o['AMT'])/o['AMT']*100)
    b = pd.DataFrame((e['CNT'] - o['CNT'])/o['CNT']*100)
    c = c.rename(columns={'AMT':'AMT_dif'})
    b = b.rename(columns={'CNT':'CNT_dif'})
    a = e.merge(c, left_index=True, right_index=True,  how='right')
    a = a.merge(b, left_index=True, right_index=True,  how='right')
    a = a[['year','month','STD_CLSS_NM','AMT','AMT_dif','CNT','CNT_dif']]
    return a.sort_values(ascending=False, by='AMT')

# 년도 별 '이용금액'과 '이용건수'의 변화량의 차이
# AMT_dif, CNT_dif : 전 년도 대비 변화량의 차이
def dif_year(x, y):
    x = x.drop(['AMT','CNT'], axis=1)
    z = y['AMT_dif'] - x['AMT_dif']
    f = y['CNT_dif'] - x['CNT_dif']
    c = pd.DataFrame(z)
    b = pd.DataFrame(f)
    x = x.rename(columns={'AMT_dif':'AMT'})
    x = x.rename(columns={'CNT_dif':'CNT'})
    a = x.merge(c, left_index=True, right_index=True,  how='right')
    a = a.merge(b, left_index=True, right_index=True,  how='right')
    a = a[['year','month','STD_CLSS_NM','AMT','AMT_dif','CNT','CNT_dif']]
    return a.sort_values(ascending=False, by='AMT') 

## 월 단위 업종 별 변동률
- p : 2019년 시도별

In [ ]:
df_q = df.groupby(['year','month','STD_CLSS_NM']).agg({'AMT':'sum'})
df_q = df_q.loc[2019]

In [ ]:
df_p = df.groupby(['year','month','CARD_SIDI_NM']).agg({'AMT':'sum'})
df_p = df_q.loc[2019]
df_p

In [ ]:
p = pd.DataFrame()
for month in range(2, 13):
    sample = (df_p.loc[month] - df_p.loc[month-1])/df_p.loc[month-1]*100
    sample['month'] = month
    p = pd.concat([p,sample],axis=0)
p

- q : 2019년 업종별

In [ ]:
q = pd.DataFrame()
for month in range(2, 13):
    sample = (df_q.loc[month] - df_q.loc[month-1])/df_q.loc[month-1]*100
    sample['month'] = month
    q = pd.concat([q,sample],axis=0)
q

-  r : 2020년

In [ ]:
df_r = df.groupby(['year','month','STD_CLSS_NM']).agg({'AMT':'sum'})
df_r = df_r.loc[2020]

In [ ]:
r = pd.DataFrame()
for month in range(2, 4):
    sample = (df_r.loc[month] - df_r.loc[month-1])/df_r.loc[month-1]*100
    sample['month']=month
    r = pd.concat([r,sample],axis=0)
r

## 시각화

In [ ]:
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')
%matplotlib inline

plt.rcParams['font.family'] = 'NanumGothic'

import matplotlib.font_manager as fm

path = 'C:/Windows/Fonts/malgun.ttf'
font_name = fm.FontProperties(fname=path, size=50).get_name()

plt.rc('font', family=font_name)

In [ ]:
f, axes = plt.subplots(9, 2, figsize=(20, 20))
i=0
j=0
for CLSS in q.index.unique():
    w = q.loc[CLSS]
    r_w = r.loc[CLSS]
    sns.lineplot(x=w.month,y=w.AMT, ax=axes[i, j]).set_title(CLSS)
    sns.lineplot(x=r.month, y= r.AMT, ax=axes[i,j]).set_title(CLSS)
    i+=1
    if i>=9:
        i=0
        j=1
plt.show()

In [ ]:
f, axes = plt.subplots(21, 2, figsize=(15, 15))
i=0
j=0
for CLSS in q.index.unique():
    w = q.loc[CLSS]
    r_w = r.loc[CLSS]
    sns.lineplot(x=w.month,y=w.AMT, ax=axes[i, j]).set_title(CLSS)
    sns.lineplot(x=r.month, y= r.AMT, ax=axes[i,j]).set_title(CLSS)
    i+=1
    if i>=21:
        i=0
        j=1
plt.show()

# 모델링

- 앞서 시행한 데이터 전처리와 EDA를 참고하여 for문을 통한 모델 구축

In [12]:
# 베이스라인
predict = pd.DataFrame()
# df = df.drop(['HOM_CCG_NM', 'CARD_CCG_NM'], axis=1)
for i,j in group.index:
    df = df1.loc[i,j] # 1번 강원-건강보조식품

    columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
    df = df.groupby(columns).sum().reset_index(drop=False) # 시군구 자동 드랍

    # 라벨 인코딩
    # 내가 간소화 함
    df_re = df.copy()
    columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE']
    for r in columns:
      encoder = LabelEncoder()
      encoded = encoder.fit(df[r])
      df_re[r] = encoded.transform(df[r])

  
    # 베이스라인
    # feature, target 설정
    train_num = df_re.sample(frac=1, random_state=0)
    x = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
    y = np.log1p(train_num['AMT'])

    try:
      k = int(len(x)*0.9)

      x_train = x[:k]
      y_train = y[:k]
      x_val = x[k:]
      y_val = y[k:]

      train_ds = lgb.Dataset(x_train, label=y_train)
      val_ds = lgb.Dataset(x_val, label=y_val)

      params = {'learning_rate' : 0.05,
                  'boosting_type': 'gbdt',
                  'objective': 'tweedie',
                  'tweedie_variance_power': 1.1,
                  'metric': 'rmse',
                  'sub_row' : 0.75,
                  'lambda_l2' : 0.1
                  }

      model = lgb.train(params,
                          train_ds,
                          1000,
                          val_ds,
                          verbose_eval = 100,
                          early_stopping_rounds = 100
                          )
    
      # 베이스라인
      # 예측 템플릿 만들기
      CARD_SIDO_NMs = df_re['CARD_SIDO_NM'].unique()
      STD_CLSS_NMs  = df_re['STD_CLSS_NM'].unique()
      HOM_SIDO_NMs  = df_re['HOM_SIDO_NM'].unique()
      AGEs          = df_re['AGE'].unique()
      SEX_CTGO_CDs  = df_re['SEX_CTGO_CD'].unique()
      FLCs          = df_re['FLC'].unique()
      years         = [2020]
      months        = [4]

      temp = []
      for CARD_SIDO_NM in CARD_SIDO_NMs:
        for STD_CLSS_NM in STD_CLSS_NMs:
          for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
              for SEX_CTGO_CD in SEX_CTGO_CDs:
                for FLC in FLCs:
                  for year in years:
                    for month in months:
                      temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])

      temp = np.array(temp)
      temp = pd.DataFrame(data=temp, columns=x.columns)

        
      pred = model.predict(temp)
      pred = np.expm1(pred)

      temp['AMT'] = np.round(pred, 0)

      temp['REG_YYMM'] = temp['year']*100 + temp['month']
      temp = temp[['REG_YYMM', 'AMT']]

      temp = temp.groupby('REG_YYMM').sum().reset_index(drop=False)

      temp['CARD_SIDO_NM'] = i
      temp['STD_CLSS_NM'] = j
      predict = predict.append(temp)
      print(i,j,"done")

    except:
      temp = pd.DataFrame()
      temp['REG_YYMM']=[202004]
      temp['CARD_SIDO_NM'] = i
      temp['STD_CLSS_NM'] = j
      temp['AMT']=0
      predict = predict.append(temp)
      print(i,j,"done")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[300]	valid_0's rmse: 0.591324
[400]	valid_0's rmse: 0.56253
[500]	valid_0's rmse: 0.535506
[600]	valid_0's rmse: 0.515659
[700]	valid_0's rmse: 0.503124
[800]	valid_0's rmse: 0.483073
[900]	valid_0's rmse: 0.476316
[1000]	valid_0's rmse: 0.474871
Did not meet early stopping. Best iteration is:
[998]	valid_0's rmse: 0.474779
경북 서양식 음식점업 done
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 0.858241
[200]	valid_0's rmse: 0.765507
[300]	valid_0's rmse: 0.72873
[400]	valid_0's rmse: 0.721356
[500]	valid_0's rmse: 0.723027
Early stopping, best iteration is:
[417]	valid_0's rmse: 0.720635
경북 수산물 소매업 done
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 1.06369
[200]	valid_0's rmse: 0.880385
[300]	valid_0's rmse: 0.816956
[400]	valid_0's rmse: 0.776362
[500]	valid_0's rmse: 0.757491
[600]	valid_0's rmse: 0.74664
[700]	valid_0's rmse: 0.741765
[800]	valid_0's rmse: 0.740447
[900]	valid_

In [13]:
pd.options.display.float_format = '{:.1f}'.format
predict

,REG_YYMM,AMT,CARD_SIDO_NM,STD_CLSS_NM
0,202004,648944525.0,강원,건강보조식품 소매업
0,202004,7231832130.0,강원,골프장 운영업
0,202004,1658780475.0,강원,과실 및 채소 소매업
0,202004,119170098.0,강원,관광 민예품 및 선물용품 소매업
0,202004,0.0,강원,그외 기타 분류안된 오락관련 서비스업
...,...,...,...,...
0,202004,1883766293.0,충북,피자 햄버거 샌드위치 및 유사 음식점업
0,202004,25947535389.0,충북,한식 음식점업
0,202004,137416177.0,충북,호텔업
0,202004,797091811.0,충북,화장품 및 방향제 소매업


# 결과 해석 및 평가

- 4월 amt 가 들어간 데이터프레임 생성
- r2score(정확도)
- rmsle(오차율)


In [14]:
data4 = pd.read_csv('202004.csv')
true = data4.groupby(['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM']).sum()['AMT']
true = true.reset_index()
true

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,88823988
1,202004,강원,골프장 운영업,4708346820
2,202004,강원,과실 및 채소 소매업,1121028924
3,202004,강원,관광 민예품 및 선물용품 소매업,14360780
4,202004,강원,그외 기타 스포츠시설 운영업,227200
...,...,...,...,...
605,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,1373635928
606,202004,충북,한식 음식점업,18911036160
607,202004,충북,호텔업,14121500
608,202004,충북,화장품 및 방향제 소매업,450507431


In [15]:
real = pd.merge(true, predict, how = 'left', on =['REG_YYMM', 'CARD_SIDO_NM','STD_CLSS_NM'] )
real = real.fillna(0)
real

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT_x,AMT_y
0,202004,강원,건강보조식품 소매업,88823988,648944525.0
1,202004,강원,골프장 운영업,4708346820,7231832130.0
2,202004,강원,과실 및 채소 소매업,1121028924,1658780475.0
3,202004,강원,관광 민예품 및 선물용품 소매업,14360780,119170098.0
4,202004,강원,그외 기타 스포츠시설 운영업,227200,29440774.0
...,...,...,...,...,...
605,202004,충북,피자 햄버거 샌드위치 및 유사 음식점업,1373635928,1883766293.0
606,202004,충북,한식 음식점업,18911036160,25947535389.0
607,202004,충북,호텔업,14121500,137416177.0
608,202004,충북,화장품 및 방향제 소매업,450507431,797091811.0


In [ ]:
from sklearn.metrics import mean_squared_log_error
np.sqrt(mean_squared_log_error(real.AMT_x, real.AMT_y))